# T5.1 Panel de control con PowerBI
## Ralphy Núñez Mercado
Hai que facer un proxecto con PowerBI que teña os seguintes elementosPower BI:

Dous informes (e vista para móbiles e ordenador por cada informe).
Alomenos 2 gráficas ou elementos gráficos ordinarios de PowerBI.
1 Mapa (aínda que sexa con outros datos sen relación).
Dúas orixes de datos, por exemplo un excel e un JSON.
Relacións entre os datos (alomenos algunha que teña sentido).
Un obxecto visual de Python.
Un orixe de datos cun script de Python.
Emprega varios orixes de datos (diferentes) para comparar.

Na práctica debe verse algo de:

Scrapping
PANDAS
Spark-HDFS <-> PowerBi

E ademáis que quede bonito o deseño do informe :)

Deberase facer con PowerBI Desktop e publicarase a app.powerbi.com coa conta @fernandowirtz.com.
Subirase a esta tarefa o arquivo .pbix. Se é moi grande para subir, darase unha ligazón ao arquivo en OneDrive (coa conta @fernandowirtz.com).
Compartirase a ligazón ao informe publicado en app.powerbi.com tamén co profe (pode poñerse no texto da tarefa ou engadir ao membro da organización co email: scj@fernandowirtz.com)

### ⬇️ Instalar las librerías

In [ ]:
!conda install pip -y || true
!conda install -c conda-forge selenium -y || true
!pip install webdriver_manager || true
!conda install pandas -y || true
!conda install sqlalchemy -y || true
!pip install pyodbc -y || true

^C


### 🦎 Instalar gecko Driver

In [ ]:
from webdriver_manager.firefox import GeckoDriverManager
GeckoDriverManager().install()

'C:\\Users\\ralphy.nunezmercado\\.wdm\\drivers\\geckodriver\\win64\\v0.36.0\\geckodriver.exe'

### ⬇️ Importar las librerías

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import re
import time
import random
import pandas as pd

### Conectar el driver de Firefox

In [ ]:
options = webdriver.FirefoxOptions()
# options.add_argument("--headless")
options.set_preference("general.useragent.override", 
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0")

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

url = "https://www.autoscout24.es/lst?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=E&source=homepage_search-mask"
driver.get(url)

# Acepta cookies si aparece el botón
try:
    btn_cookies = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar') or contains(., 'Aceptar todas') or contains(., 'Aceptar todo')]"))
    )
    btn_cookies.click()
    print("Cookies aceptadas")
    time.sleep(1)
except Exception:
    print("No apareció banner de cookies o ya estaba aceptado")

datos_coches = []
pagina = 1

while True:
    print(f"Extrayendo página {pagina}...")

    try:
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@data-testid="list-item"]')))
    except Exception:
        print("No se encontraron coches en la página, saliendo del bucle.")
        break

    coches = driver.find_elements(By.XPATH, '//article[@data-testid="list-item"]')

    for articulo in coches:
        try:
            marca = articulo.get_attribute('data-make') or ''
            modelo = articulo.get_attribute('data-model') or ''
            precio = articulo.get_attribute('data-price') or ''

            try:
                km_text = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-mileage_road"]').text
                km = re.sub(r'[^\d]', '', km_text)
            except:
                km = ''

            try:
                year_text = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-calendar"]').text
                year = year_text.split('/')[-1] if '/' in year_text else year_text
            except:
                year = ''

            try:
                combustible = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-gas_pump"]').text
            except:
                combustible = ''

            try:
                cv_text = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-speedometer"]').text
                match = re.search(r'\((\d+)\s*CV\)', cv_text)
                cv = match.group(1) if match else ''
            except:
                cv = ''

            try:
                ubicacion_raw = articulo.find_element(By.XPATH, './/*[@data-testid="sellerinfo-address"]').text
                if "Contáctanos en:" in ubicacion_raw:
                    ubicacion = ubicacion_raw.split()[-1]
                else:
                    ubicacion = ubicacion_raw

                # Extraer código postal (ej: 3760 → 03760, ES-50013 → 50013)
                match_cp = re.search(r'\b(?:ES-)?(\d{4,5})\b', ubicacion_raw)
                if match_cp:
                    codigo_postal = match_cp.group(1).zfill(5)
                else:
                    codigo_postal = ''
            except:
                ubicacion = ''
                codigo_postal = ''

            try:
                transmision = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-transmission"]').text
            except:
                transmision = ''

            datos_coches.append([
                marca.capitalize(), modelo, precio, km, year,
                combustible, cv, ubicacion, transmision, codigo_postal
            ])
        except Exception as e:
            print(f"Error extrayendo un coche: {e}")
            continue

    if url.startswith("file://"):
        break

    time.sleep(random.uniform(2, 4))
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(random.uniform(1, 2))

    try:
        paginacion = driver.find_element(By.XPATH, '//*[@data-testid="listpage-pagination"]')
        next_btn_li = paginacion.find_element(By.XPATH, ".//li[contains(@class, 'prev-next') and not(contains(@class, 'previous'))]")
        next_btn = next_btn_li.find_element(By.TAG_NAME, "button")
        if next_btn.get_attribute("aria-disabled") == "true":
            print("No hay más páginas.")
            break

        # Esperar que se actualice la página
        ultimo_articulo = coches[-1]
        next_btn.click()
        pagina += 1
        wait.until(EC.staleness_of(ultimo_articulo))
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@data-testid="list-item"]')))
    except Exception as e:
        print("No hay más páginas o no se encontró el botón de siguiente.")
        break

# Guardar en CSV
with open('coches.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([
        'Marca', 'Modelo', 'Precio', 'KM', 'Año',
        'Tipo_de_combustible', 'CV', 'Ubicacion', 'Transmision', 'Codigo_postal'
    ])
    writer.writerows(datos_coches)

print(f"¡Datos guardados en coches.csv! Total coches: {len(datos_coches)}")
#driver.quit()


Cookies aceptadas
Extrayendo página 1...
Extrayendo página 2...
Extrayendo página 3...
Extrayendo página 4...
Extrayendo página 5...
Extrayendo página 6...
Extrayendo página 7...
Extrayendo página 8...
Extrayendo página 9...
Extrayendo página 10...
Extrayendo página 11...
Extrayendo página 12...
Extrayendo página 13...
Extrayendo página 14...
Extrayendo página 15...
Extrayendo página 16...
Extrayendo página 17...
Extrayendo página 18...
Extrayendo página 19...
Extrayendo página 20...
No hay más páginas.
¡Datos guardados en coches.csv! Total coches: 400


In [ ]:
import time
import random
import re
import csv
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager

def cerrar_overlays(driver):
    overlays_xpaths = [
        "//button[contains(@class, 'modal-box__header__close')]",
        "//button[contains(@class, 'close')]",
        "//button[contains(@aria-label, 'Cerrar')]",
        "//button[contains(@aria-label, 'Close')]",
        "//button[contains(@id, 'save-search-list-button')]"
    ]
    for xpath in overlays_xpaths:
        try:
            close_btn = driver.find_element(By.XPATH, xpath)
            if close_btn.is_displayed():
                close_btn.click()
                time.sleep(1)
        except Exception:
            pass


options = webdriver.FirefoxOptions()
options.set_preference("general.useragent.override", 
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0")

driver = webdriver.Firefox(service=FirefoxService(GeckoDriverManager().install()), options=options)
wait = WebDriverWait(driver, 20)

url = "https://www.autoscout24.es/lst?sort=standard&desc=0&ustate=N%2CU&atype=C&cy=E&source=homepage_search-mask"
driver.get(url)

# Acepta cookies si aparece el botón
try:
    btn_cookies = wait.until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar') or contains(., 'Aceptar todas') or contains(., 'Aceptar todo')]"))
    )
    btn_cookies.click()
    print("Cookies aceptadas")
    time.sleep(1)
except Exception:
    print("No apareció banner de cookies o ya estaba aceptado")

datos_coches = []

# 1. Haz click en el input para mostrar todas las sugerencias de marcas
wait.until(EC.element_to_be_clickable((By.ID, "make-input-primary-filter"))).click()
time.sleep(1)

# 2. Recoge todas las marcas del autocompletar (solo para obtener la lista de nombres)
wait.until(EC.presence_of_element_located((By.XPATH, '//ul[@id="make-input-primary-filter-suggestions"]')))
sugerencias = driver.find_elements(By.XPATH, '//ul[@id="make-input-primary-filter-suggestions"]/li[@role="option"]')

marcas = []
for sug in sugerencias:
    marca = sug.text.strip()
    if marca and not "divider" in sug.get_attribute("class"):
        marcas.append(marca)

print(f"Total marcas encontradas: {len(marcas)}")

for idx, marca_nombre in enumerate(marcas):
    print(f"\n==> Procesando marca: {marca_nombre} ({idx+1}/{len(marcas)})")
    driver.get(url)
    wait.until(EC.element_to_be_clickable((By.ID, "make-input-primary-filter"))).click()
    time.sleep(1)
    wait.until(EC.presence_of_element_located((By.XPATH, '//ul[@id="make-input-primary-filter-suggestions"]')))
    marca_encontrada = False
    try:
        sugerencias_actuales = driver.find_elements(By.XPATH, '//ul[@id="make-input-primary-filter-suggestions"]/li[@role="option"]')
        for sug in sugerencias_actuales:
            if sug.text.strip().lower() == marca_nombre.lower():
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", sug)
                time.sleep(0.5)
                driver.execute_script("arguments[0].click();", sug)
                time.sleep(1)
                # Espera y cierra el modal de login si aparece
                for _ in range(4):
                    if driver.find_elements(By.XPATH, "//button[contains(@class, 'modal-box__header__close')]"):
                        cerrar_overlays(driver)
                        break
                    time.sleep(0.5)
                marca_encontrada = True
                break
        if not marca_encontrada:
            print(f"No se pudo seleccionar la marca: {marca_nombre}")
            continue
    except Exception as e:
        print(f"No se pudo seleccionar la marca: {marca_nombre} - {e}")
        continue



    # Espera a que se recargue la lista de coches
    try:
        wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@data-testid="list-item"]')))
    except Exception:
        print("No se encontraron coches para esta marca.")
        continue

    pagina = 1
    while pagina <= 20:
        print(f"Extrayendo página {pagina} de {marca_nombre}...")

        try:
            wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@data-testid="list-item"]')))
        except Exception:
            print("No se encontraron coches en la página, saliendo del bucle.")
            break

        coches = driver.find_elements(By.XPATH, '//article[@data-testid="list-item"]')

        for articulo in coches:
            try:
                marca = articulo.get_attribute('data-make') or marca_nombre
                modelo = articulo.get_attribute('data-model') or ''
                precio = articulo.get_attribute('data-price') or ''

                try:
                    km_text = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-mileage_road"]').text
                    km = re.sub(r'[^\d]', '', km_text)
                except:
                    km = ''

                try:
                    year_text = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-calendar"]').text
                    year = year_text.split('/')[-1] if '/' in year_text else year_text
                except:
                    year = ''

                try:
                    combustible = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-gas_pump"]').text
                except:
                    combustible = ''

                try:
                    cv_text = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-speedometer"]').text
                    match = re.search(r'\((\d+)\s*CV\)', cv_text)
                    cv = match.group(1) if match else ''
                except:
                    cv = ''

                try:
                    ubicacion_raw = articulo.find_element(By.XPATH, './/*[@data-testid="sellerinfo-address"]').text
                    if "Contáctanos en:" in ubicacion_raw:
                        ubicacion = ubicacion_raw.split()[-1]
                    else:
                        ubicacion = ubicacion_raw

                    match_cp = re.search(r'\b(?:ES-)?(\d{4,5})\b', ubicacion_raw)
                    if match_cp:
                        codigo_postal = match_cp.group(1).zfill(5)
                    else:
                        codigo_postal = ''
                except:
                    ubicacion = ''
                    codigo_postal = ''

                try:
                    transmision = articulo.find_element(By.XPATH, './/*[@data-testid="VehicleDetails-transmission"]').text
                except:
                    transmision = ''

                datos_coches.append([
                    marca.capitalize(), modelo, precio, km, year,
                    combustible, cv, ubicacion, transmision, codigo_postal
                ])
            except Exception as e:
                print(f"Error extrayendo un coche: {e}")
                continue

        # Intentar ir a la siguiente página (máximo 20)
        try:
            paginacion = driver.find_element(By.XPATH, '//*[@data-testid="listpage-pagination"]')
            next_btn_li = paginacion.find_element(By.XPATH, ".//li[contains(@class, 'prev-next') and not(contains(@class, 'previous'))]")
            next_btn = next_btn_li.find_element(By.TAG_NAME, "button")
            if next_btn.get_attribute("aria-disabled") == "true":
                print("No hay más páginas para esta marca.")
                break

            ultimo_articulo = coches[-1]
            next_btn.click()
            pagina += 1
            wait.until(EC.staleness_of(ultimo_articulo))
            wait.until(EC.presence_of_all_elements_located((By.XPATH, '//article[@data-testid="list-item"]')))
            time.sleep(random.uniform(2, 4))
        except Exception as e:
            print("No hay más páginas o no se encontró el botón de siguiente para esta marca.")
            break

# Guardar en CSV
with open('coches.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow([
        'Marca', 'Modelo', 'Precio', 'KM', 'Año',
        'Tipo_de_combustible', 'CV', 'Ubicacion', 'Transmision', 'Codigo_postal'
    ])
    writer.writerows(datos_coches)

print(f"¡Datos guardados en coches.csv! Total coches: {len(datos_coches)}")
driver.quit()


Cookies aceptadas
Total marcas encontradas: 269

==> Procesando marca: Mercedes-Benz (1/269)
Extrayendo página 1 de Mercedes-Benz...
Extrayendo página 2 de Mercedes-Benz...
Extrayendo página 3 de Mercedes-Benz...
Extrayendo página 4 de Mercedes-Benz...
Extrayendo página 5 de Mercedes-Benz...
Extrayendo página 6 de Mercedes-Benz...
Extrayendo página 7 de Mercedes-Benz...
Extrayendo página 8 de Mercedes-Benz...
Extrayendo página 9 de Mercedes-Benz...
Extrayendo página 10 de Mercedes-Benz...
Extrayendo página 11 de Mercedes-Benz...
Extrayendo página 12 de Mercedes-Benz...
Extrayendo página 13 de Mercedes-Benz...
Extrayendo página 14 de Mercedes-Benz...
Extrayendo página 15 de Mercedes-Benz...
Extrayendo página 16 de Mercedes-Benz...
Extrayendo página 17 de Mercedes-Benz...
Extrayendo página 18 de Mercedes-Benz...
Extrayendo página 19 de Mercedes-Benz...
Extrayendo página 20 de Mercedes-Benz...
No hay más páginas para esta marca.

==> Procesando marca: BMW (2/269)
Extrayendo página 1 de BM

In [ ]:

cp_to_comunidad = {
    '01': 'País Vasco',
    '02': 'Andalucía',
    '03': 'Comunidad Valenciana',
    '04': 'Andalucía',
    '05': 'Andalucía',
    '06': 'Extremadura',
    '07': 'Baleares',
    '08': 'Cataluña',
    '09': 'Castilla y León',
    '10': 'Extremadura',
    '11': 'Andalucía',
    '12': 'Comunidad Valenciana',
    '13': 'Castilla-La Mancha',
    '14': 'Andalucía',
    '15': 'Galicia',
    '16': 'Castilla-La Mancha',
    '17': 'Cataluña',
    '18': 'Andalucía',
    '19': 'Castilla-La Mancha',
    '20': 'País Vasco',
    '21': 'Andalucía',
    '22': 'Aragón',
    '23': 'Andalucía',
    '24': 'Castilla y León',
    '25': 'Cataluña',
    '26': 'La Rioja',
    '27': 'Galicia',
    '28': 'Madrid',
    '29': 'Andalucía',
    '30': 'Región de Murcia',
    '31': 'Navarra',
    '32': 'Galicia',
    '33': 'Asturias',
    '34': 'Castilla y León',
    '35': 'Canarias',
    '36': 'Galicia',
    '37': 'Castilla y León',
    '38': 'Canarias',
    '39': 'Cantabria',
    '40': 'Castilla y León',
    '41': 'Andalucía',
    '42': 'Castilla y León',
    '43': 'Cataluña',
    '44': 'Aragón',
    '45': 'Castilla-La Mancha',
    '46': 'Comunidad Valenciana',
    '47': 'Castilla y León',
    '48': 'País Vasco',
    '49': 'Castilla y León',
    '50': 'Aragón',
    '51': 'Ceuta',
    '52': 'Melilla'
}

def obtener_comunidad(codigo_postal):
    if pd.isna(codigo_postal):
        return "Desconocida"
    codigo_postal = str(codigo_postal).zfill(5)  # Asegura 5 dígitos
    prefijo = codigo_postal[:2]
    return cp_to_comunidad.get(prefijo, "Desconocida")

# Leer el CSV y forzar que 'Codigo_Postal' se lea como texto
df = pd.read_csv("coches.csv", dtype={"Codigo_postal": str})

# Aplicar la función
df['Comunidad_autonoma'] = df['Codigo_postal'].apply(obtener_comunidad)

# Guardar el nuevo archivo
df.to_csv("coches.csv", index=False)

print("Archivo generado: coches.csv")


Archivo generado: coches.csv
